In [1]:
import pandas as pd
import numpy as np
from sklearn import preprocessing, svm
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split

import quandl
import math

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


In [2]:
df = quandl.get('WIKI/GOOGL')

df = df[['Adj. Open', 'Adj. Close','Adj. High','Adj. Low','Adj. Volume']]
df['HL_PCT'] = (df['Adj. High'] - df['Adj. Close']) / df['Adj. Close'] * 100
df['PCT_Change'] = (df['Adj. Close']-df['Adj. Open'])/df['Adj. Open'] * 100

df = df[['Adj. Close','HL_PCT', 'PCT_Change', 'Adj. Volume']]

df.head()

,Adj. Close,HL_PCT,PCT_Change,Adj. Volume
Date,,,,
2004-08-19,50.322842,3.712563,0.324968,44659000.0
2004-08-20,54.322689,0.710922,7.227007,22834300.0
2004-08-23,54.869377,3.729433,-1.227880,18256100.0
2004-08-24,52.597363,6.417469,-5.726357,15247300.0
2004-08-25,53.164113,1.886792,1.183658,9188600.0


In [3]:
forecast_col = 'Adj. Close'
forecast_out = math.ceil(0.1*len(df)) # 10 percent ahead data is used as forecasted output

df['label'] = df[forecast_col].shift(-forecast_out) # Future data of Adj. Close is used as Label Column of the Present value
df.dropna(inplace=True)

df.head()

,Adj. Close,HL_PCT,PCT_Change,Adj. Volume,label
Date,,,,,
2004-08-19,50.322842,3.712563,0.324968,44659000.0,214.005615
2004-08-20,54.322689,0.710922,7.227007,22834300.0,210.725489
2004-08-23,54.869377,3.729433,-1.227880,18256100.0,208.072299
2004-08-24,52.597363,6.417469,-5.726357,15247300.0,218.288837
2004-08-25,53.164113,1.886792,1.183658,9188600.0,223.309335


In [4]:
forecast_out # means the value of column label is 343 row value of Column Adj. Close

343

In [5]:
df.iloc[0, 4] # value of 0 row data of column Label

214.00561483153143

In [6]:
df.iloc[343, 0] # value of 343 row data of column Adj. Close
# ans for both the columns is same. 

214.00561483153143

## APPLY Linear Regression

In [7]:
X = df.drop(['label'],1).values

# X = np.array(df.drop(['label'],1))
# df.drop(['label'],1) is Dataframe either use .values to convert it to array or np.array(df.drop(['label'],1))

y = df['label'].values
X

/var/folders/kv/fgspkbq1315gy0xwcyk56_z80000gn/T/ipykernel_5469/4271996660.py:1: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  X = df.drop(['label'],1).values


array([[ 5.03228418e+01,  3.71256291e+00,  3.24967503e-01,
         4.46590000e+07],
       [ 5.43226889e+01,  7.10922353e-01,  7.22700723e+00,
         2.28343000e+07],
       [ 5.48693765e+01,  3.72943327e+00, -1.22788010e+00,
         1.82561000e+07],
       ...,
       [ 8.11980000e+02,  5.00012316e-01,  1.24060197e+00,
         1.76906900e+06],
       [ 8.05590000e+02,  7.59691655e-01,  4.68927329e-01,
         3.09851000e+06],
       [ 7.80290000e+02,  3.81524818e+00, -3.66790123e+00,
         5.90960900e+06]])

In [8]:
X = preprocessing.scale(X)
X

array([[-1.57627789,  2.24965149,  0.22714733,  4.29480253],
       [-1.55512222, -0.33929392,  4.68260107,  1.70252919],
       [-1.55223073,  2.26420234, -0.77525789,  1.15874413],
       ...,
       [ 2.452217  , -0.52120596,  0.8182142 , -0.79953667],
       [ 2.41841953, -0.2972299 ,  0.3200773 , -0.6416296 ],
       [ 2.28460484,  2.33821857, -2.35035768, -0.30773557]])

In [9]:
len(X)

3081

In [10]:
len(y)

3081

In [11]:
#split dataset
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2)

In [12]:
# choose model
model = LinearRegression()

In [13]:
#train model
model.fit(X_train, y_train)

LinearRegression()

In [14]:
# finding accuracy
accuracy = model.score(X_train, y_train)

In [15]:
accuracy

0.8794906170451542

In [18]:
#apply different model

model = svm.SVR()
model.fit(X_train, y_train)
accuracy = model.score(X_train, y_train)
accuracy

0.7162460182290298

In [19]:
accuracy

0.7162460182290298